#### Name: Noha Magdy Ali 
#### ID: 1610126
# Age and Gender Recognition using openCV and CNNs 

### Needed packages 

In [7]:
from skimage import data, io
from matplotlib import pyplot as plt
import numpy as np
from skimage import img_as_ubyte,img_as_float
from skimage import data, io, filters
from matplotlib.pyplot import imshow, show
from skimage.morphology import watershed
from skimage.measure import label
from skimage.color import label2rgb
from skimage import feature
from skimage.filters import threshold_otsu,  threshold_local, sobel
from skimage.morphology import erosion,dilation,opening,closing,square,disk
from skimage.measure import label
from skimage.color import label2rgb
import os
import pandas as pd
import tensorflow as tf
import functools
!pip install mitdeeplearning
import mitdeeplearning as mdl
from tqdm import tqdm
import cv2

### Loading and Preparing Data

In [13]:
path_to_training_data = tf.keras.utils.get_file('/Dataset', 'https://drive.google.com/drive/u/0/folders/0BxYys69jI14kSVdWWllDMWhnN2c', extract=True)


 188416/Unknown - 0s 2us/step

In [15]:
data= os.listdir("/Dataset")

Data = pd.DataFrame(columns=["Image", "Age", "Gender"])


for idx,i in zip(range(len(data)),data):
    age, gender, race, name=i.split("_")
    img = io.imread("Dataset/"+i)
    Data.loc[idx] = [ img, age, gender]
    #append(pd.Series(data={"Image":img, "Age":age, "Gender":gender}, name=idx))

NotADirectoryError: ignored

In [0]:
Data.shape

(3251, 3)

In [0]:
Data.head()

,Image,Age,Gender
0,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...",100,1
1,"[[[187, 152, 156], [187, 152, 156], [187, 152,...",10,0
2,"[[[177, 230, 234], [176, 229, 235], [173, 226,...",10,0
3,"[[[233, 230, 223], [234, 231, 224], [234, 231,...",10,0
4,"[[[216, 221, 224], [216, 221, 224], [216, 221,...",10,1


In [0]:
rand_idxs = np.random.choice(Data.shape[0], 500, True)
test = Data.loc[rand_idxs]
train = Data
train.drop(rand_idxs)

,Image,Age,Gender
0,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...",100,1
1,"[[[187, 152, 156], [187, 152, 156], [187, 152,...",10,0
2,"[[[177, 230, 234], [176, 229, 235], [173, 226,...",10,0
4,"[[[216, 221, 224], [216, 221, 224], [216, 221,...",10,1
5,"[[[253, 251, 254], [253, 251, 254], [253, 251,...",10,1
...,...,...,...
3245,"[[[251, 223, 201], [251, 223, 201], [251, 223,...",99,1
3247,"[[[121, 216, 86], [119, 214, 84], [118, 213, 8...",99,1
3248,"[[[118, 115, 98], [115, 112, 95], [114, 111, 9...",99,1
3249,"[[[133, 61, 2], [133, 61, 2], [134, 62, 3], [1...",9,0


In [0]:
train.head()

,Image,Age,Gender
0,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...",100,1
1,"[[[187, 152, 156], [187, 152, 156], [187, 152,...",10,0
2,"[[[177, 230, 234], [176, 229, 235], [173, 226,...",10,0
3,"[[[233, 230, 223], [234, 231, 224], [234, 231,...",10,0
4,"[[[216, 221, 224], [216, 221, 224], [216, 221,...",10,1


In [0]:
test.head()

,Image,Age,Gender
2332,"[[[250, 229, 140], [250, 228, 142], [252, 230,...",50,0
1353,"[[[223, 227, 213], [223, 227, 213], [223, 227,...",32,0
2976,"[[[71, 142, 134], [71, 142, 134], [71, 142, 13...",67,0
222,"[[[16, 27, 33], [32, 43, 49], [45, 56, 62], [4...",23,0
103,"[[[197, 199, 194], [221, 226, 222], [239, 245,...",21,0


In [0]:
n_filters = 12 

def classifier(n_outputs=1):
    Conv2D = functools.partial(tf.keras.layers.Conv2D, padding='same', activation='relu')
    BatchNormalization = tf.keras.layers.BatchNormalization
    Flatten = tf.keras.layers.Flatten
    Dense = functools.partial(tf.keras.layers.Dense, activation='relu')

    model = tf.keras.Sequential([
    Conv2D(filters=1*n_filters, kernel_size=5,  strides=2),
    BatchNormalization(),
    
    Conv2D(filters=2*n_filters, kernel_size=5,  strides=2),
    BatchNormalization(),

    Conv2D(filters=4*n_filters, kernel_size=3,  strides=2),
    BatchNormalization(),

    Conv2D(filters=6*n_filters, kernel_size=3,  strides=2),
    BatchNormalization(),

    Flatten(),
    Dense(512),
    Dense(n_outputs, activation=None),
  ])
    return model
My_classifier = classifier()

In [9]:
# Training hyperparameters
batch_size = 32
num_epochs = 2  # keep small to run faster
learning_rate = 5e-4

optimizer = tf.keras.optimizers.Adam(learning_rate) # define our optimizer
loss_history = mdl.util.LossHistory(smoothing_factor=0.99) # to record loss evolution
plotter = mdl.util.PeriodicPlotter(sec=2, scale='semilogy')
if hasattr(tqdm, '_instances'): tqdm._instances.clear() # clear if it exists

@tf.function
def standard_train_step(x, y):
    with tf.GradientTape() as tape:
    # feed the images into the model
        logits = standard_classifier(x) 
    # Compute the loss
        loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=logits)

  # Backpropagation
    grads = tape.gradient(loss, standard_classifier.trainable_variables)
    optimizer.apply_gradients(zip(grads, standard_classifier.trainable_variables))
    return loss

# The training loop!
for epoch in range(num_epochs):
    for idx in tqdm(range(loader.get_train_size()//batch_size)):
        # Grab a batch of training data and propagate through the network
        x, y = loader.get_batch(batch_size)
        loss = standard_train_step(x, y)

        # Record the loss and plot the evolution of the loss as a function of training
        loss_history.append(loss.numpy().mean())
        plotter.plot(loss_history.get())

NameError: ignored